In [6]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import joblib
from experta import Fact, KnowledgeEngine, Rule, DefFacts
from sklearn.tree import export_graphviz
import sys
import os

# Add the rule_based_system directory to the Python path
sys.path.append('../rule_based_system')

# Import the rules
from rules import HeartDiseaseRules

# Load cleaned data
df = pd.read_csv("../data/cleaned_data.csv")

# Split data (ensure you're using the same test set as in decision_tree_model.ipynb)
X = df.drop("target", axis=1)
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load Decision Tree model
model = joblib.load("../ml_model/decision_tree_model.pkl")

# Evaluate model on the entire dataset (as in your original model_comparison.ipynb)
y_pred_full = model.predict(X)
full_accuracy = accuracy_score(y, y_pred_full)
print(f"Decision Tree Accuracy on full dataset: {full_accuracy}")

# Save original comparison results (as in your original file)
original_results = pd.DataFrame({
    "Model": ["Decision Tree"],
    "Accuracy": [full_accuracy]
})
original_results.to_csv("../reports/accuracy_comparison.csv", index=False)

# Now evaluate the Decision Tree on test set
y_pred = model.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred)
dt_report = classification_report(y_test, y_pred)

print("\nDecision Tree Model Results on test set:")
print(f"Accuracy: {dt_accuracy}")
print("Classification Report:")
print(dt_report)

# Run rule-based system on the same test set
rule_results = []
for i, row in X_test.reset_index(drop=True).iterrows():
    # Convert normalized features back to original scale for rules
    age = row['age'] * 100  # Assuming normalization as in data_processing
    cholesterol = row['chol'] * 300 + 100  # Based on your min-max scaling
    
    # Apply rules (simplified example)
    engine = HeartDiseaseRules()
    engine.reset()
    engine.declare(Fact(age=age))
    engine.declare(Fact(cholesterol=cholesterol))
    # Add other facts as needed
    engine.run()
    
    # Get result - corrected to properly handle Experta facts
    # Convert facts to a more easily inspectable format
    fact_list = [f for f in engine.facts.values()]
    
    # Debug information - uncomment if having issues with facts
    # print(f"Row {i}, Facts count: {len(fact_list)}")
    # for fact in fact_list:
    #     print(f"Fact type: {type(fact)}")
    #     print(f"Fact dir: {dir(fact)}")
    #     print(f"Fact values: {str(fact)}")
    #     print("---")
    
    # Check if any fact contains 'risk' with value 'high'
    has_high_risk = False
    for fact in fact_list:
        if hasattr(fact, '__dict__') and hasattr(fact, '__factid__'):
            # This is a proper Fact object
            # Access its attributes - in Experta, facts store their data as attributes
            if hasattr(fact, 'risk') and fact.risk == 'high':
                has_high_risk = True
                break
    
    rule_results.append(1 if has_high_risk else 0)

# Calculate metrics for rule-based system
rule_accuracy = accuracy_score(y_test, rule_results)
rule_report = classification_report(y_test, rule_results)

print("\nRule-based System Results:")
print(f"Accuracy: {rule_accuracy}")
print("Classification Report:")
print(rule_report)

# Compare the two approaches
comparison = pd.DataFrame({
    'Model': ['Decision Tree', 'Rule-based System'],
    'Accuracy': [dt_accuracy, rule_accuracy]
})

# Save comparison results
comparison.to_csv("../reports/model_comparison.csv", index=False)
print("\nModel Comparison:")
print(comparison)

# Explainability analysis
# Export the decision tree structure to DOT format
try:
    # Visualize decision tree
    best_model = model  # Using the loaded model as the best model
    dot_data = export_graphviz(
        best_model, 
        out_file="../reports/decision_tree.dot",  # Save DOT file directly
        feature_names=X.columns,
        class_names=['No Disease', 'Disease'],
        filled=True
    )
    print("Decision tree DOT file saved to '../reports/decision_tree.dot'")
    
    # Try to use graphviz if available
    try:
        import graphviz
        graph = graphviz.Source(dot_data)
        graph.render("../reports/decision_tree_visualization")
        print("Decision tree visualization rendered successfully")
    except Exception as e:
        print(f"Graphviz rendering failed: {e}")
        print("To visualize the decision tree, please install Graphviz from https://graphviz.org/download/")
        print("And make sure it's added to your system PATH")
except Exception as e:
    print(f"Error exporting decision tree: {e}")

# Write a report on explainability comparison
with open("../reports/explainability_comparison.txt", "w") as f:
    f.write("Decision Tree vs. Rule-based System Explainability\n")
    f.write("================================================\n\n")
    f.write("Decision Tree:\n")
    f.write(f"- Number of nodes: {best_model.tree_.node_count}\n")
    f.write(f"- Maximum depth: {best_model.tree_.max_depth}\n")
    f.write("- Decisions based on learned patterns in data\n\n")
    f.write("Rule-based System:\n")
    f.write("- Number of rules: [count your rules]\n")
    f.write("- Rules defined by human experts\n")
    f.write("- Transparent decision logic\n\n")
    f.write("Comparison:\n")
    f.write("- Decision tree provides better performance metrics\n") 
    f.write("- Rule-based system offers more intuitive explanations\n")
    # Add more insights based on your specific analysis

Decision Tree Accuracy on full dataset: 0.9970731707317073

Decision Tree Model Results on test set:
Accuracy: 0.9853658536585366
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       102
           1       1.00      0.97      0.99       103

    accuracy                           0.99       205
   macro avg       0.99      0.99      0.99       205
weighted avg       0.99      0.99      0.99       205


Rule-based System Results:
Accuracy: 0.4975609756097561
Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.66       102
           1       0.00      0.00      0.00       103

    accuracy                           0.50       205
   macro avg       0.25      0.50      0.33       205
weighted avg       0.25      0.50      0.33       205


Model Comparison:
               Model  Accuracy
0      Decision Tree  0.985366
1  Rule-based System  0.497561
D

c:\Users\hanaa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanaa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanaa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif